In [ ]:
#CNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [ ]:
# Data path
data_path = "/content/sample_data/ship_dataset"

In [ ]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((600, 416)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(3, 3))]),
    transforms.ToTensor()
])

In [ ]:
# Create a dataset
train_dataset = ImageFolder(root=data_path + '/train', transform=transform)
test_dataset = ImageFolder(root=data_path + '/test', transform=transform)
valid_dataset = ImageFolder(root=data_path + '/valid', transform=transform)

In [ ]:
# Create a DataLoader 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
# Model description
class ShipClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(ShipClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 150 * 104, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 150 * 104)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Create the model
model = ShipClassifier(num_classes=10)

In [ ]:
# Loss function and optimization function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# training cycle
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

print("Training completed.")

# Save model
pretrained_model_path = "/content/sample_data/pretrained_model.pth"
torch.save(model.state_dict(), pretrained_model_path)

In [ ]:
# Fine Tuning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm
from PIL import Image

In [ ]:
# Old model
pretrained_model_path = "/content/sample_data/pretrained_model.pth"
pretrained_model = torch.load(pretrained_model_path)

In [ ]:
# Defining the new dataset and updating the number of classes
new_data_path = "/content/sample_data/new_dataset"
new_dataset = ImageFolder(root=new_data_path, transform=transform)
num_classes = len(new_dataset.classes)

In [ ]:
# Updating the output layer of the old model
pretrained_model.fc2 = nn.Linear(512, num_classes)

# Create the new model and optimization function
model = pretrained_model
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# training cycle
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

print("Training completed.")

In [ ]:
# Test accuracy

In [ ]:
model.eval()

In [ ]:
correct = 0
total = 0

In [ ]:
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [ ]:
accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')